In [1]:
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np
import ipyvolume as ipv
from numpy.linalg import norm as norm
from scipy.optimize import  minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [60]:
from berny import Berny, geomlib, coords,Geometry

In [2]:
HF=gto.M(atom="F 0 0 0;H 0 0 1.695741",unit="bohr",
            basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])})
mfhf = scf.RHF(HF)
mfhf.scf()

converged SCF energy = -100.065540168732


-100.06554016873241

In [3]:
dm=mfhf.make_rdm1()

In [5]:
def Vne(coords,mol=HF,dm=dm):
    mol.set_rinv_orig_(coords)
    dV=mol.intor('int1e_rinv')
    return -np.einsum('ij,ij',dV,dm)

In [69]:
dL=[-1,0,0,0] #-1 charge central atom
def Vnn(coords,mol=HF):
    Vnn=0
    for i in range (mol.natm):
        Vnn+= (mol.atom_charge(i)+dL[i])/np.linalg.norm(coords-mol.atom_coord(i))
    return Vnn

In [7]:
Vnn([0,.5,-1.3],HF)

6.072918030765468

In [9]:
def Vtot(coords,mol=HF,dm=dm):
    return Vnn(coords,mol)+Vne(coords,mol,dm)


In [11]:
def Vtot_fl(angle,mol=HF,dm=dm):
    coords=np.asarray([0,np.sin(angle),np.cos(angle)])*1.695741
    return Vtot(coords,mol=HF,dm=dm)

In [45]:
prot_angle=minimize(Vtot_fl,1.8).x[0]
prot_angle*180/np.pi

119.57807446314582

In [46]:
prot_angle

2.0870311125768453

In [47]:
prot_site=np.array([0.,np.sin(prot_angle),np.cos(prot_angle)])*1.695741
prot_site

array([ 0.        ,  1.47475862, -0.83703319])

In [139]:
prot_site=minimize(Vtot,[0,.5,-1.3]).x
norm(prot_site),np.arccos(prot_site[2]/norm(prot_site))*180/np.pi

(1.551075762750464, 118.42168311645113)

In [140]:
def dVne(coords,mol=HF,dm=dm):
    mol.set_rinv_orig_(coords)
    vrinv = -mol.intor('int1e_iprinv', comp=3)
    vrinv += vrinv.transpose(0,2,1)
    return np.einsum('xij,ij->x',vrinv,dm)

In [141]:
dVne([0,.5,-1.3])

array([ 9.86049182e-14,  1.38257692e+00, -3.72835980e+00])

In [142]:
Vne([0,.5,-1.3],HF,dm)-Vne([0.,.5,-1.301],HF,dm)

-0.0037267647027503514

In [143]:
#fig=plt.figure(figsize=(20, 10))
##ax = plt.axes(projection=’3d’)
#ax =fig.add_subplot(111, projection='3d')
#ax.plot_surface(X, Y, Z, rstride=1, cstride=1,cmap="terrain" ,edgecolor=None)
#ax.set(xlabel='x', ylabel='y', zlabel='Energy',title="Probing PES for protonation")

In [144]:
#from matplotlib import cm
#colormap = cm.coolwarm
#znorm = Z - Z.min()
#znorm /= znorm.ptp()
#znorm.min(), znorm.max()
#color = colormap(znorm)

In [145]:
#ipv.figure()
#mesh = ipv.plot_surface(X, Y, 100*Z, color=color[...,:3])
#ipv.show()

In [146]:
prot_site

array([ 7.47877000e-07,  1.36412230e+00, -7.38245477e-01])

# How to develop the basis functions or do none.


In [147]:
pHF=FcM(fcs=[0,0,-1],atom="F 0 0 0;H 0 0 1.695741; H {} {} {}".format(prot_site[0],prot_site[1],prot_site[2]),unit="bohr",
          basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])},charge=1,verbose=4)
pHF.charge=0

System: uname_result(system='Linux', node='ubuntudesk', release='4.15.0-134-generic', version='#138-Ubuntu SMP Fri Jan 15 10:52:18 UTC 2021', machine='x86_64', processor='x86_64')  Threads 4
Python 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
numpy 1.16.2  scipy 1.2.1
Date: Tue Mar 16 17:31:58 2021
PySCF version 1.7.5.1
PySCF path  /home/administrator/anaconda3/lib/python3.7/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 10
[INPUT] charge = 1
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT]  1 F      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr
[INPUT]  2 H      0.000000000000   0.000000000000   0.897347492823 AA    0.000000000000   0.000000000000   1.695741000000 Bohr
[INPUT]  3 H      0.000000395759   0.721862431884  -0.390662682271 AA    0.000000747877   1.364122295874  -0.738245476581

In [148]:
pHF.atom_charges(),pHF.nelec

(array([9., 1., 0.]), (5, 5))

In [149]:
pHFmf=scf.RHF(pHF)
#pHFmf.build()

In [150]:
pHFmf.mo_occ=np.zeros(pHF.nao)
pHFmf.mo_occ[:HF.nao]=mfhf.mo_occ
pHFmf.mo_energy=np.zeros(pHF.nao)
pHFmf.mo_energy[:HF.nao]=mfhf.mo_energy
pHFmf.mo_coeff=np.zeros([pHF.nao,pHF.nao])
pHFmf.mo_coeff[:HF.nao,:HF.nao]=mfhf.mo_coeff

In [151]:
pHFmf.make_rdm1()[:HF.nao,:HF.nao]-dm

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00, -4.3790577e-47]])

In [152]:
af=aaff(pHFmf,[-1,0,1])+alc_deriv_grad_nuc(pHF,[-1,0,1])

In [153]:
phfc=pHF.atom_coords()*bohr2ang
pbg=Geometry(['O','H','H'],phfc)
b=Berny(pbg,debug=True)

In [154]:
b.send((mfhf.e_tot,af))

{'geom': <Geometry 'H2O'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 2, angles: 1, dihedrals: 0">,
 'H': array([[0.54502214, 0.        , 0.        ],
        [0.        , 0.58393728, 0.        ],
        [0.        , 0.        , 0.23574722]]),
 'weights': array([1.21116031, 1.2976384 , 1.12068171]),
 'future': Point(q=array([1.72184667, 1.84993781, 2.06685717]), E=None, g=None),
 'first': False,
 'interpolated': Point(q=array([1.695741  , 1.55107576, 2.06684828]), E=-100.06554016873241, g=array([-3.52923448e-02, -4.15661736e-01, -9.46906798e-06])),
 'predicted': Point(q=array([1.72184674, 1.84993775, 2.06685736]), E=-100.16442307891194, g=None),
 'previous': Point(q=array([1.695741  , 1.55107576, 2.06684828]), E=-100.06554016873241, g=array([-3.52923448e-02, -4.15661736e-01, -9.46906798e-06])),
 'best': Point(q=arr

In [64]:
q=np.array([1.72184679, 1.84993778, 2.06685607])  # bd H1_O,bd H2_O, angle (H-O-H) -- Bohr,radians

In [65]:
norm(phfc[1])*ang2bohr,norm(phfc[2])*ang2bohr,np.arccos((phfc[1].dot(phfc[2])/norm(phfc[1])/norm(phfc[2])))

(1.6957409999672617, 1.6957409999672617, 2.0870311125768453)

In [66]:
pbg.coords,phfc

(array([[ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.89734749],
        [ 0.        ,  0.78040865, -0.44293889]]),
 array([[ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.89734749],
        [ 0.        ,  0.78040865, -0.44293889]]))

# H$_2$O

In [67]:
OH2=gto.M(atom="O 0 0 0;H 0 -0.15229 1.76907; H 0   1.7401 -0.35319",unit="bohr",
            basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mfoh2 = scf.RHF(OH2)
mfoh2.scf()
dmoh2=mfoh2.make_rdm1()

converged SCF energy = -76.0633193329876


In [161]:
ps_h2o=minimize(Vtot,np.array([1.5,0,0]),args=(OH2,dmoh2)).x

In [167]:
pH2O=FcM(fcs=[0,0,0,-1],atom="O 0 0 0;H 0 -0.15229 1.76907; H 0 1.7401 -0.35319; H {} {} {}".format(*ps_h2o),\
         unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])},charge=1)
pH2O.charge=0

In [125]:
pH2Omf=scf.RHF(pH2O)
pH2Omf.mo_occ=np.zeros(pH2O.nao)
pH2Omf.mo_occ[:OH2.nao]=mfoh2.mo_occ
pH2Omf.mo_energy=np.zeros(pH2O.nao)
pH2Omf.mo_energy[:OH2.nao]=mfoh2.mo_energy
pH2Omf.mo_coeff=np.zeros([pH2O.nao,pH2O.nao])
pH2Omf.mo_coeff[:OH2.nao,:OH2.nao]=mfoh2.mo_coeff

In [133]:
afph2o=aaff(pH2Omf,[-1,0,0,1])+alc_deriv_grad_nuc(pH2O,[-1,0,0,1])

In [137]:
ph2oc=pH2O.atom_coords()*bohr2ang
pbg2=Geometry(['N','H','H','H'],ph2oc)
b2=Berny(pbg2,debug=True)
b2.send((mfoh2.e_tot,afph2o))

{'geom': <Geometry 'H3N'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 3, angles: 3, dihedrals: 0">,
 'H': array([[0.53257882, 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.53258653, 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.57335329, 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.21010689, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.2261895 ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.22619277]]),
 'weights': array([1.18350849, 1.18352562, 1.27411843, 1.14117516, 1.09421186,
        1.09421435]),
 'future': Point(q=array([1.81528413, 1.81525491, 1.91249103, 1.87018295,

#     NH$_3$

In [156]:
nh3optg,oh2optg,fhoptg=np.load("optgeoms.npy")
nh3lst=[]
for x in range(4):
    nh3lst.append("NHHH"[x])
    nh3lst.extend(nh3optg[x].tolist()) 
nh3srt=(("{} "*4+";")*4)[:-1].format(*nh3lst)
NH3=gto.M(atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf3 = scf.RHF(NH3)
mf3.scf()

converged SCF energy = -56.2221835947889


-56.2221835947889

In [157]:
nh3optg

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.37164299,  1.84861891],
       [ 1.52728421,  0.80129301, -0.76149538],
       [-1.52728421,  0.80129301, -0.76149538]])

In [159]:
nh3dm=mf3.make_rdm1()
ps_nh3=minimize(Vtot,np.array([0,-1.5,.3]),args=(NH3,nh3dm)).x

In [166]:
pNH3=FcM(fcs=[0,0,0,0,-1],atom=nh3srt+"; H {} {} {} ".format(*ps_nh3),unit="bohr",\
         basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},charge=1)
pNH3.charge=0

In [168]:
pNH3mf=scf.RHF(pNH3)
pNH3mf.mo_occ=np.zeros(pNH3.nao)
pNH3mf.mo_occ[:NH3.nao]=mf3.mo_occ
pNH3mf.mo_energy=np.zeros(pNH3.nao)
pNH3mf.mo_energy[:NH3.nao]=mf3.mo_energy
pNH3mf.mo_coeff=np.zeros([pNH3.nao,pNH3.nao])
pNH3mf.mo_coeff[:NH3.nao,:NH3.nao]=mf3.mo_coeff

In [169]:
afpnh3=aaff(pNH3mf,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3,[-1,0,0,0,1])

In [171]:
pnh3c=pNH3.atom_coords()*bohr2ang
pbg3=Geometry(['C','H','H','H','H'],pnh3c)
b3=Berny(pbg3,debug=True)
b3.send((mf3.e_tot,afpnh3))

{'geom': <Geometry 'CH4'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 4, angles: 6, dihedrals: 0">,
 'H': array([[0.51366855, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.5137292 , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.5137292 , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.55378188, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.19547151,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.    

In [175]:
b3._state.coords._coords

[Bond(0, 1, weak=0),
 Bond(0, 2, weak=0),
 Bond(0, 3, weak=0),
 Bond(0, 4, weak=0),
 Angle(1, 0, 2, weak=0),
 Angle(1, 0, 3, weak=0),
 Angle(1, 0, 4, weak=0),
 Angle(2, 0, 3, weak=0),
 Angle(2, 0, 4, weak=0),
 Angle(3, 0, 4, weak=0)]